<a href="https://colab.research.google.com/github/TeamMAMI/MAMI/blob/overlaid_text_removal/overlaid_text_removal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pasaentuciudad.com.mx/remove-text-from-images-using-cv2-and-keras-ocr/

#Don't run the last code block until you've checked which directory you're in

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install keras-ocr

In [3]:
import cv2
import math
import glob
import keras_ocr
import os, os.path
import numpy as np
import pandas as pd
from pandas import read_csv
from tqdm import tqdm
from PIL import Image as pil_image
from numpy import asarray
import matplotlib.pyplot as plt
from IPython.display import Image as ip_image

Define the midpoint and inpaint text functions

In [4]:
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)

In [5]:
 def inpaint_text(img_path, pipeline):
    # read image
    img = img_path

    # generate (word, box) tuples 
    prediction_groups = pipeline.recognize([img])
    mask = np.zeros(img.shape[:2], dtype="uint8")
    for box in tqdm(prediction_groups[0]):
        x0, y0 = box[1][0]
        x1, y1 = box[1][1] 
        x2, y2 = box[1][2]
        x3, y3 = box[1][3] 
        
        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)
        
        thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))
        
        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255,    
        thickness)
        img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)

    return(img)

make the pipeline

In [6]:
pipeline = keras_ocr.pipeline.Pipeline()

Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


make a list of all the image names in the trial set

In [7]:
# df = pd.DataFrame(data=read_csv('trial.csv', sep='\t'))

# trial_images = list(set(df["file_name"].tolist()))
# print(sorted(trial_images))
# print(len(trial_images))

...or do this for the training set

In [8]:
%cd drive/Shareddrives/team_MAMI/MAMI/TRAIN 
!pwd
!ls

/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN
/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN
Cleaned_Images	Images	training.csv


In [9]:

df = pd.DataFrame(data=read_csv('training.csv', sep='\t'))

train_images = list(set(df["file_name"].tolist()))
print(sorted(train_images))
print(len(train_images))

['1.jpg', '10.jpg', '1000.jpg', '10000.jpg', '10006.jpg', '10007.jpg', '10008.jpg', '10009.jpg', '1001.jpg', '10010.jpg', '10011.jpg', '10012.jpg', '10013.jpg', '10014.jpg', '10016.jpg', '10017.jpg', '10018.jpg', '1002.jpg', '10021.jpg', '10022.jpg', '10023.jpg', '10025.jpg', '10027.jpg', '10028.jpg', '10029.jpg', '1003.jpg', '10030.jpg', '10032.jpg', '10033.jpg', '10034.jpg', '10035.jpg', '10036.jpg', '10037.jpg', '10038.jpg', '10039.jpg', '10040.jpg', '10042.jpg', '10043.jpg', '10044.jpg', '10045.jpg', '10046.jpg', '10048.jpg', '10049.jpg', '1005.jpg', '10050.jpg', '10052.jpg', '10053.jpg', '10055.jpg', '10056.jpg', '10057.jpg', '10059.jpg', '1006.jpg', '10060.jpg', '10061.jpg', '10063.jpg', '10065.jpg', '10066.jpg', '10067.jpg', '10068.jpg', '10069.jpg', '10070.jpg', '10071.jpg', '10072.jpg', '10073.jpg', '10075.jpg', '10076.jpg', '10077.jpg', '10078.jpg', '10079.jpg', '10080.jpg', '10081.jpg', '10083.jpg', '10085.jpg', '10086.jpg', '10087.jpg', '10088.jpg', '10089.jpg', '10090.jpg'

In [10]:
# %mkdir Cleaned_Images

In [11]:
%cd drive/Shareddrives/team_MAMI/MAMI/TRIAL
!pwd
!ls

[Errno 2] No such file or directory: 'drive/Shareddrives/team_MAMI/MAMI/TRIAL'
/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN
/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN
Cleaned_Images	Images	training.csv


#Pause here and make sure you're in the right directory

In [12]:
!pwd
!ls

/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN
Cleaned_Images	Images	training.csv


Loop through the list of image names, opening, processing, and saving as pngs

In [13]:
# trial_images_path = "./Images/"
# for img in tqdm(trial_images):
  # image = asarray(cv2.imread(trial_images_path + img))
  # hold = inpaint_text(image, pipeline)
  # img_num = img.split('.')[0]
  # cv2.imwrite(f'/content/drive/Shareddrives/team_MAMI/MAMI/TRIAL/Clean_images/{img_num}.png',hold)

... or do this for the training images

but first get a list of the unprocessed images

In [14]:
%cd Cleaned_Images
!pwd

/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN/Cleaned_Images
/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN/Cleaned_Images


In [16]:
cleaned_images = os.listdir()
print(len(cleaned_images))

10000


In [17]:
train_nums = [name.split('.')[0] for name in train_images]
print(train_nums)
print(len(train_nums))
print(len(train_images))
cleaned_nums = [name.split('.')[0] for name in cleaned_images]
print(cleaned_nums)
print(len(cleaned_nums))
print(len(cleaned_images))

['6930', '3154', '5743', '3523', '11334', '4790', '10121', '9299', '5705', '13291', '7220', '4446', '10518', '9488', '6256', '3596', '882', '1485', '10417', '1020', '9409', '7561', '562', '6252', '11117', '1342', '3336', '5904', '6079', '9379', '761', '5905', '2209', '4584', '11650', '10167', '8984', '10826', '9522', '7486', '2578', '8227', '10561', '5151', '1144', '3085', '11595', '2746', '3843', '6367', '4902', '3722', '3834', '8730', '7324', '11164', '1012', '6311', '8036', '5528', '8784', '5944', '4290', '8879', '2837', '8547', '8234', '271', '5812', '7049', '469', '8144', '11688', '4526', '6143', '3016', '4115', '8808', '7149', '11722', '9055', '5575', '7836', '8104', '4928', '8109', '6044', '10387', '6983', '850', '926', '4931', '10087', '6795', '11480', '1554', '10860', '1058', '1980', '2298', '11720', '2595', '7578', '9760', '10239', '4720', '11014', '4241', '8618', '906', '10666', '5763', '4999', '8993', '462', '5049', '9375', '11073', '6107', '10825', '11013', '4352', '8897',

In [18]:
unprocessed = list(set(train_nums) - set(cleaned_nums))

print(len(unprocessed))

0


In [ ]:
%cd ..
!pwd
!ls

In [ ]:
train_images_path = "./Images/"
for img in tqdm(unprocessed):
  image = asarray(cv2.imread(train_images_path + img + '.jpg'))
  hold = inpaint_text(image, pipeline)
  # img_num = img.split('.')[0]
  cv2.imwrite(f'/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN/Cleaned_Images/{img}.png',hold)